Preliminaries
==============

Mount your GDrive using the file browser OR run the code below:

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

The current Google Colab environment comes pre-loaded with an older version of the gensim package, which we'll use to interrogate an embeddings model.  So we'll also need to update gensim in our environment to the latest version.

In [ ]:
!pip install --upgrade gensim

In [14]:
import pandas as pd

Word Embeddings
===============

Our sessions so far have worked off the idea of document annotation to produce new, sometimes highly useful metdata about texts. We've used this information for everything from information retrieval tasks to predictive classification. Along the way, we've also made some passing discussions about how such annotations work to quantify or identify the semantics of those tasks (our work with POS tags, for example). But what we haven't yet done is produce a model of semantic meaning ourselves. This is another core task of NLP, and there are several different ways to approach building a statistical representation of tokens' meanings. The present section discusses one of the most popular methods of doing so: **word embeddings** also known as **word vectors**. Below, we'll overview what word embeddings are, demonstrate how to build and use them, talk about important considerations regarding bias, and apply all this to a document clustering task.

The corpus we'll use is Melanie Walsh’s [collection] of ~380 obituaries from the _New York Times_, which we also used in our TF-IDF text vectoring and classifiation and Topic Modeling sessions. Our return to it here is meant to chime with that discussion, for word embeddings enable us to perform a similar kind of text vectorization. Though, as we'll discuss, the resultant vectors will be considerably more feature-rich than what we could achieve with tf-idf alone.

[collection]: https://melaniewalsh.github.io/Intro-Cultural-Analytics/00-Datasets/00-Datasets.html
[in the context of tf-idf scores]: https://ucdavisdatalab.github.io/workshop_getting_started_with_textual_data/05_clustering-and-classification.html

```{admonition} Learning objectives
By the end of this chapter, you will be able to:
+ Explain what word embeddings are
+ Use `gensim` to train and load word embeddings models
+ Identify and analyze word relationships in these models
+ Recognize how bias can inhere in embeddings
+ Encode documents with a word embeddings model
```

Word Embeddings: Introduction
-------------------------------------

The word-embedding technique was first introduced in 2013, and it spawned a host of different variants that completely flooded the field of NLP until about 2018. In part, word embedding's popularity stems from the relatively simple intuition behind it, which is known as the **distributional hypothesis**: "a word is characterized by the company it keeps" (Firth). Words that appear in similar contexts, in other words, have similar meanings, and what word embeddings do is represent that context-specific information through a set of features. As a result, similar words share similar data representations, and we can leverage that similarity to explore the semantic space of a corpus, to encode documents with feature-rich data, and more.

If you're familiar with [tf-idf vectors], the underlying data structure of word embeddings is the same: every word is represented by a vector of _n_ features. But a key difference lies in the **sparsity** of the vectors – or, in the case of word embeddings, the _lack_ of sparsity. As we saw in the last chapter, tf-idf vectors can suffer from the [curse of dimensionality], something that's compounded by the fact that such vectors must contain features for every word in corpus, regardless of whether a document has that word. This means tf-idf vectors are highly sparse: they contain many 0s. Word embeddings, on the other hand, do not. They're what we call **dense** representations. Each one is a fixed-length, non-sparse vector (of 50-300 dimensions, usually) that is much more information-rich than tf-idf. As a result, embeddings tend to be capable of representing more nuanced relationships between corpus words – a performance improvement that is further boosted by the fact that many of the most popular models had the advantage of being trained on billions and billions of tokens.

The other major difference between these vectors and tf-idf lies in how the former are created. While at root, word embeddings represent token co-occurence data (just like a document-term matrix), they are the product of millions of guesses made by a neural network. Training this network involves making predictions about a target word, based on that word's context. We are not going to delve into the math behind these predictions (though [this post] does); however, it is worth noting that there are two different training set ups for a word embedding model:

```{margin} For more on CBOW vs. skip-gram
Check out this blog post, [Words as Vectors].

[Words as Vectors]: https://iksinc.online/tag/continuous-bag-of-words-cbow/
```

1. **Common Bag of Words (CBOW)**: given a window of words on either side of a target, the network tries to predict what word the target should be
2. **Skip-grams**: the network starts with the word in the middle of a window and picks random words within this window to use as its prediction targets

As you may have noticed, these are just mirrored versions of one another. CBOW starts from context, while skip-gram tries to rebuild context. Regardless, in both cases the network attempts to maximize the likelihood of its predictions, updating its weights accordingly over the course of training. Words that repeatedly appear in similar contexts will help shape thse weights, and in turn the model will associate such words with similar vector representations. If you'd like to see all this in action, Xin Rong has produced a [fantastic, interactive visualization] of how word embedding models learn.

Of course, the other way to understand how word embeddings work is to use them yourself. We'll move on to doing so now.

[Transformer]: https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)
[tf-idf vectors]: https://ucdavisdatalab.github.io/workshop_getting_started_with_textual_data/04_corpus-analytics.html#weighted-metrics-tf-idf-scores
[curse of dimensionality]: https://en.wikipedia.org/wiki/Curse_of_dimensionality
[this post]: https://medium.com/analytics-vidhya/maths-behind-word2vec-explained-38d74f32726b
[fantastic, interactive visualization]: https://ronxin.github.io/wevi/

Loading the Data
--------------------

Before we begin working with word embeddings in full, let's load a corpus manifest file, which will help us keep track of all the obituaries.

In [ ]:
import pandas as pd

manifest = pd.read_csv('/content/drive/MyDrive/rbs_digital_approaches_2022/nlp_workshop_data/session_three/manifest.csv', index_col=0)
manifest = manifest.assign(YEAR = pd.to_datetime(manifest['YEAR'], format='%Y').dt.year)

print(
    f"Number of obituaries: {len(manifest)}",
    f"\nDate range: {manifest['YEAR'].min()}--{manifest['YEAR'].max()}"
)

In [ ]:
manifest.groupby('YEAR').count().plot(
    figsize=(15, 5),
    y='NAME',
    title='Obituaries per Year',
    ylabel='Num. obituaries',
    xlabel='Year',
    legend=False
);

Here's a sampling of the corpus:

In [ ]:
for idx in manifest.sample(10).index:
    name, date = manifest.loc[idx, 'NAME'], manifest.loc[idx, 'YEAR']
    print(f"{name} ({date})")

Now we can load the obituaries themselves. For demonstration perposes, each of the files in the corpus have already been processed by the text cleaning pipeline that we instituted in our ealier sessions on tf-idf and topic modeling: they represent the lowercase, stopped, and lemmatized versions of the originals.

No extra loading considerations are needed here either. We'll just use `glob` to get our file paths and iterate through the list, loading each document into a `corpus` list. Also note that here, for demonstration purposes, we are forgoing the use of a file manifest and that we still must split the file contents.

In [4]:
import glob

paths = glob.glob('/content/drive/MyDrive/rbs_digital_approaches_2022/nlp_workshop_data/session_three/obits/*.txt')
paths.sort()

corpus = []
for path in paths:
    with open(path, 'r') as fin:
        doc = fin.read()
        doc = doc.split()
        corpus.append(doc)

With this done, we can move on to the model.

Using an Embeddings Model
----------------------------------

At this point, we are at a crossroads. On the one hand, we could train a word embeddings model using our corpus documents as is. The `gensim` library offers functionality for this, and it's a relatively easy operation. On the other, we could use premade embeddings, which are usually trained on a more general – and much larger – set of documents. There is a tradeoff here:

+ Training a corpus-specific model will more faithfully represent the token behavior of the texts we'd like to analyze, but these representations could be _too_ specific, especially if the model doesn't have enough data to train on; the resultant embeddings may be closer to topic models than to word-level semantics
+ Using premade embeddings gives us the benefit of generalization: the vectors will cleave more closely to how we understand language; but such embeddings might a) miss out on certain nuances we'd like to capture, or b) introduce biases into our corpus (more on this below)

In our case, the decision is difficult. With this corpus, a model trained on the obituaries alone does not produce vectors that can fully demonstrate the capabilities of the word embedding technique. The corpus is just a little too specific and too small. We could've used a larger corpus, but doing so would introduce slow-downs in the workshop session. Because of this, for the analytic portions of this suession we wil use a premade model, in this case, the Stanford [GloVe] embeddings (the 200-dimension version). GloVe was trained on billions of tokens, spanning Wikipedia data, newswire articles, even Twitter.

That said, going with GloVe introduces its own problems. For one thing, we can't show you how to train a word embeddings model itself – at least not live. The code to do so, however, is reproduced below:

```{margin} Model parameters
There are many different parameters to select from in `gensim`. You can find them in the [Word2Vec documentation].

[Word2Vec documentation]: https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
```

```python
from gensim.models import Word2Vec

n_dimensions = 100
model = Word2Vec(n_dimensions)
model.build_vocab(corpus)
model.train(corpus, total_words=model.corpus_total_words, epochs=5)
```

Another problem has to do with the data GloVe was trained on. It's so large that we can't account for all the content, and this becomes particularly detrimental when it comes to bias. [Researchers have found] that general embeddings models reproduce gender-discriminatory language, even hate speech, by virtue of the fact that they are trained on huge amounts of text data, often without consideration of whether the content of such data is something one would endorse. GloVe is [known to be biased] in this way. We'll show an example later on in this chapter and will discuss this in much more detail during our live session, but for now just note that the effects of bias _do_ shape how we represent our corpus, and it's important to keep an eye out for this when working with the data.

[GloVe]: https://nlp.stanford.edu/projects/glove/
[Researchers have found]: https://www.technologyreview.com/2016/07/27/158634/how-vector-space-mathematics-reveals-the-hidden-sexism-in-language/
[known to be biased]: http://arxiv.org/abs/1607.06520

### Loading a model

With all that said, we can move on. Below, we load GloVe embeddings into our workspace using a `gensim` wrapper.

In [1]:
from gensim.models import KeyedVectors

model = KeyedVectors.load('~/estc_vectors.kv')

The `KeyedVectors` object acts almost like a dictionary. You can do certain Python operations directly on it, like using `len()` to find the number of tokens in the model.

In [2]:
n_tokens = len(model)

print(f"Number of unique tokens in the model: {n_tokens:,}")

Number of unique tokens in the model: 44,721


### Token mappings

Each token in the model (what `gensim` calls a "key") has an associated index. This mapping is accessible via the `.key_to_index` attribute:

In [3]:
model.key_to_index

{'the': 0,
 'of': 1,
 'and': 2,
 'in': 3,
 'to': 4,
 'a': 5,
 'The': 6,
 'by': 7,
 'for': 8,
 '...': 9,
 'A': 10,
 'By': 11,
 'at': 12,
 'on': 13,
 'with': 14,
 'from': 15,
 'his': 16,
 'is': 17,
 'which': 18,
 'or': 19,
 'that': 20,
 'an': 21,
 'be': 22,
 'all': 23,
 'Mr.': 24,
 'An': 25,
 'their': 26,
 'as': 27,
 'are': 28,
 'To': 29,
 'John': 30,
 'or,': 31,
 '&c.': 32,
 'In': 33,
 'With': 34,
 'this': 35,
 'His': 36,
 'upon': 37,
 'late': 38,
 'most': 39,
 'Lord': 40,
 'other': 41,
 'de': 42,
 '&': 43,
 'several': 44,
 'William': 45,
 'new': 46,
 'our': 47,
 'being': 48,
 'against': 49,
 'account': 50,
 'into': 51,
 'day': 52,
 'Thomas': 53,
 'it': 54,
 'two': 55,
 'who': 56,
 'year': 57,
 'And': 58,
 'some': 59,
 'one': 60,
 'present': 61,
 'said': 62,
 'letter': 63,
 'English': 64,
 'before': 65,
 'St.': 66,
 '-': 67,
 'Right': 68,
 'sermon': 69,
 'was': 70,
 'true': 71,
 'Sir': 72,
 'now': 73,
 'Honourable': 74,
 'not': 75,
 'first': 76,
 'made': 77,
 'King': 78,
 'great': 79,
 

If you want to get the vector representation for a token, you can use either the key or the index. The syntax is just like a Python dictionary. Below, we randomly select a single token from the model vocabulary's `.index_to_key` attribute and find the index associated with it.

In [4]:
import random

rand_token = random.choice(model.index_to_key)
rand_idx = model.key_to_index[rand_token]

print(f"The index position for '{rand_token}' is {rand_idx}")

The index position for 'church-catechism;' is 40423


Here's its vector:

In [5]:
model.vectors[rand_idx]

array([-5.70862927e-03, -1.33481305e-02,  3.23085226e-02,  6.97142631e-03,
       -8.73323157e-03, -1.82167850e-02, -2.02227123e-02,  4.75008823e-02,
       -3.52476686e-02,  5.56792133e-02, -1.88121181e-02, -3.16595496e-03,
       -2.33052410e-02,  1.32035399e-02, -8.42893124e-03,  2.55607534e-02,
        1.28857484e-02, -1.82746295e-02, -7.65345199e-03, -5.82167739e-03,
        2.67808959e-02,  1.68807115e-02,  2.29667220e-02,  4.80142869e-02,
       -1.07718380e-02, -1.15012722e-02, -1.66429076e-02,  3.72506641e-02,
        2.45385095e-02, -3.82035933e-02, -6.11247635e-03,  1.02562960e-02,
       -2.24740021e-02, -6.63561048e-03, -3.76092866e-02,  1.14537291e-02,
        6.43050969e-02, -1.74017120e-02,  1.80914402e-02,  1.42833712e-02,
       -5.01399338e-02, -9.75505740e-04,  1.33385211e-02,  3.35457809e-02,
       -1.00062881e-02,  1.75217865e-03, -4.22832780e-02,  1.25392862e-02,
        2.75270566e-02, -1.98554262e-04,  5.39634097e-03,  3.77082005e-02,
       -4.58236504e-03, -

Finally, we can store the entire model vocabulary in a `set` and show a few examples of the tokens therein.

In [6]:
model_vocab = set(model.index_to_key)

for token in random.sample(model_vocab, 10):
    print(token)

Urry,
T.M.
trials,
I.R.
Clarkson,
P.B.
burnt
expressing
Bulls,
Israel:


You may find some unexpected tokens in this output. Though it has been ostensibly trained on an English corpus, GloVe contains multilingual text. It also contains lots of noisy tokens, which range from erroneous segmentations ("drummer/percussionist" is one token, for example) to password-like strings and even HTML markup. Depending on your task, you may not notice these tokens, but they do in fact influence the overall shape of the model, and sometimes you'll find them cropping up when you're hunting around for similar terms and the like (more on this soon).

### Out-of-vocabulary tokens

While GloVe's vocabulary sometimes seems _too_ expansive, there are other instances where it's too restricted.

In [7]:
assert 'esparza' in model, "Not in vocabulary!"

AssertionError: Not in vocabulary!

In [8]:
assert 'unshaped' in model, "Not in vocabulary!"

AssertionError: Not in vocabulary!

If the model wasn't trained on a particular word, it won't have a corresponding vector for that word either. This is crucial. Because models like GloVe only know what they've been trained on, you need to be aware of any potential discrepancies between their vocabularies and your corpus data. If you don't keep this in mind, sending unseen, or **out-of-vocabulary**, tokens to GloVe will throw errors in your code:

In [9]:
model['esparza']

KeyError: "Key 'esparza' not present"

In [10]:
model['unshaped']

KeyError: "Key 'unshaped' not present"

There are a few ways to handle this problem. The most common is to simply _not encode_ tokens in your corpus that don't have a corresponding vector in GloVe. Below, we construct three dictionaries for our corpus data. The first contains all tokens, while the second and third are comprised of tokens that are and are not in Glove, respectively. We identify whether the model has a token using its `.has_index_for()` method.

In [11]:
corpus_vocab = set(token for doc in corpus for token in doc)
in_glove = set(token for token in corpus_vocab if model.has_index_for(token))
no_glove = set(token for token in corpus_vocab if model.has_index_for(token) == False)

print(
    f"Total words in the corpus vocabulary: {len(corpus_vocab):,}",
    f"\nNumber of corpus words in GloVe: {len(in_glove):,}",
    f"\nNumber of corpus words not in GloVe: {len(no_glove):,}"
)

NameError: name 'corpus' is not defined

Any subsequent code we write will need to reference these dictionaries to determine whether it should encode a token.

While this is what we'll indeed do below, obviously it isn't an ideal situation. But it's one of the consequences of using premade models. There are, however, a few other ways to handle out-of-vocabulary terms. Some models offer special "UNK" tokens, which you could associate with all of your problem tokens. This, at the very least, enables you to have _some_ representation of your data. A more complex approach involves taking the mean embedding of the word vectors surrounding an unknown token; and depending on the model, you can also train it further, adding extra tokens from your domain-specific text. Instructions for this last option are available [here] in the `gensim` documentation.

[here]: https://radimrehurek.com/gensim/models/word2vec.html#usage-examples

Word relationships
----------------------

Later on we'll use GloVe to encode our corpus texts. But before we do, it's worth demonstrating more generally some of the properties of word vectors. Vector representations of text allow us to perform various mathematical operations on our corpus that approximate (though maybe _only_ approximate) semantics. The most common among these operations is finding the **cosine similarity** between two vectors. Our Getting Started with Textual Data series has a whole [chapter] on this measure, so if you haven't encountered it before, we recommend you read that. But in short: cosine similarity measures the difference between vectors' orientation in a feature space (here, the feature space is comprised of each of the vectors' 200 dimentions). The closer two vectors are, the more likely they are to share semantic similarities.

[chapter]: https://ucdavisdatalab.github.io/workshop_getting_started_with_textual_data/05_clustering-and-classification.html#

### Cosine similarity

`gensim` provides easy access to this measure and other such vector space operations, and we can use this functionality to explore relationships between words in a model. To find the cosine similarity between the vectors for two words in GloVe, simply use the model's `.similarity()` method:

In [12]:
a, b = 'god', 'king'
sim = model.similarity(a, b)

print(f"Consine similarity score for '{a}' and '{b}': {sim:0.4f}")

Consine similarity score for 'god' and 'king': 0.5709


The only difference between the score above and the one that you might produce, say, with `scikit-learn`'s cosine similarity implementation is that `gensim` bounds its values from `[-1,1]`, whereas the latter uses a `[0,1]` scale. While in `gensim` it's still the case that similar words score closer to `1`, highly dissimilar words will be closer to `-1`.

At any rate, we can get the top _n_ most similar words for a word using `.most_similar()`. The function defaults to 10 entries, but you can change that with the `topn` paramter.

In [18]:
targets = ['good', 'evil']

for token in targets:
    similarities = model.most_similar(token)
    print(f"Tokens most similar to '{token}':")
    df = pd.DataFrame(similarities, columns=['WORD', 'SCORE'])
    display(df)
    print("")

Tokens most similar to 'good':


,WORD,SCORE
0,"Duty,",0.862411
1,necessity,0.848188
2,with,0.843640
3,"God,",0.842049
4,...,0.831795
5,dealing,0.822570
6,July,0.815165
7,Perkins,0.812576
8,religion,0.812063
9,allen,0.808958


Tokens most similar to 'evil':


,WORD,SCORE
0,duorum,0.968976
1,Bess,0.963792
2,140,0.962226
3,emergencies,0.958172
4,Amelia,0.956763
5,messuages,0.956489
6,treasury,0.954779
7,vain,0.953120
8,Italienne,0.952619
9,Taylor.,0.952215


We can also find the _least_ similar word. This is useful to show, because it pressures our idea of what counts as similarity. Mathematical similarity does not always align with concepts like synonyms and antonyms. For example, it's probably safe to say that the semantic opposite of "good" – that is, its antonym – is "evil." But in the world of vector spaces, the least similar word to "good" is:

In [26]:
targets = ['good', 'evil']

for token in targets:
    model.most_similar(positive=["good"])

In [27]:
model.most_similar(negative=["good"])

[('Ludgate', 0.6743211150169373),
 ('rejoycing', 0.6687424778938293),
 ('usefulness', 0.6529909372329712),
 ('Ulick,', 0.634185791015625),
 ('Filson.', 0.6178230047225952),
 ('Gilt', 0.6175533533096313),
 ('printed.)', 0.6150988936424255),
 ('concurrent', 0.6084148287773132),
 ('universities', 0.6026992797851562),
 ('Ane', 0.5950279831886292)]

In [33]:
targets = ['good', 'evil']
for token in targets:
    similarities = model.most_similar(token, topn=len(model))
    print(f"Tokens least similar to '{token}':")
    df = pd.DataFrame(similarities, columns=['WORD', 'SCORE'])
    display(df.tail(10))
    print("")

Tokens least similar to 'good':


,WORD,SCORE
44710,Ane,-0.595028
44711,universities,-0.602699
44712,concurrent,-0.608415
44713,printed.),-0.615099
44714,Gilt,-0.617553
44715,Filson.,-0.617823
44716,"Ulick,",-0.634186
44717,usefulness,-0.652991
44718,rejoycing,-0.668742
44719,Ludgate,-0.674321



Tokens least similar to 'evil':


,WORD,SCORE
44710,soldier,-0.626211
44711,vvorke,-0.627029
44712,councell,-0.629111
44713,expence.,-0.630666
44714,Teachers,-0.639093
44715,peninsula,-0.651329
44716,check,-0.655280
44717,"Burnham,",-0.659340
44718,revealed:,-0.662371
44719,Thunder,-0.765463


Now we can also compute the overall cosine similarity between the terms 'good' and 'evil'

In [34]:
a, b = 'good', 'evil'
sim = model.similarity(a, b)

print(f"Consine similarity score for {a} and {b}: {sim:0.4f}")

Consine similarity score for good and evil: 0.2287


How do we make sense of this? Well, it has to do with the way the word embeddings are created. Since embeddings models are ultimately trained on co-occurrence data, words that tend to appear in similar kinds of contexts will be more similar in a mathematical sense than those that don't.

### Visualizing the vector space

One way to start getting a feel for all this is to visualize the word vectors. We do so below by sampling a portion of the GloVe vectors and then reducing them into two-dimensional data, which we can plot. First, let's build two functions.

```{margin} How we create the visualization data
`sample_embeddings()` takes a sample from GloVe:

1. First it randomly selects indices in the model
2. Then it uses these to subset the vectors
3. Finally it associates the tokens with their respective indices to produce a set of labels

`prepare_vis_data()` takes the sampled vectors and their labels and reduces them with a t-SNE embedder

1. The `TSNE()` portion of the code does the work of reducing our 200-dimension vectors into only two dimensions
2. Then the function converts the two-dimensional data into a dataframe and associates the labels
```

In [35]:
from sklearn.manifold import TSNE

def sample_embeddings(vectors, samp=1000):
    n_vectors = vectors.shape[0]
    mask = random.sample(range(n_vectors), samp)
    vectors = vectors[mask]
    vocab = [model.index_to_key[idx] for idx in mask]
    
    return vectors, vocab

def prepare_vis_data(vectors, labels):
    reduced = TSNE(
        n_components=2,
        learning_rate='auto',
        init='random',
        random_state=357
    ).fit_transform(vectors)
    
    vis_data = pd.DataFrame(reduced, columns=['X', 'Y'])
    vis_data['LABEL'] = labels
    
    return vis_data

Now we can retrieve all the vectors from GloVe using the `.key_to_index` attribute. With those stored in a `numpy` array, it's time to sample them and create the visualization data.

In [36]:
import numpy as np
all_vectors = np.array([model[idx] for idx in model.key_to_index])

sampled, sampled_vocab = sample_embeddings(all_vectors)
vis_data = prepare_vis_data(sampled, sampled_vocab)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

```{margin} This visualization is partial
Since we've run a t-SNE reduction on the sampled embeddings, the graph layout only takes into account the relative differences between that sampling, not all of GloVe. Going the latter route would take a while to compute – there would be 400,000 embeddings to reduce!
```

With the reduced embeddings made, it's time to plot them. Have a look around at the results. What seems right to you? What surprises you?

In [ ]:
import altair as alt

alt.Chart(vis_data).mark_circle(size=30).encode(
    x='X',
    y='Y',
    tooltip='LABEL'
).properties(
    height=650,
    width=650
).interactive()

### Other relationships

Beyond cosine similarity, there are other word relationships to explore via vector space math. For example, one way of modeling something like a _concept_ is to think about what other concepts comprise it. In other words: what plus what creates a new concept? Could we identify concepts by adding together vectors to create a new vector? Which words would this new vector be closest to in the vector space? Using the `.similar_by_vector()` method, we can find out.

```{margin} What this loop does
For each concept in our `concepts` dictionary:

1. Get its associated pair of words
2. Query the model for those words' vectors and add them together to create a new vector
3. Find the most similar words to this new vector
4. Use a dataframe to display the results
```

In [38]:
concepts = {'love': ('hate', 'marriage')}

for concept in concepts:
    pair = concepts[concept]
    generated_concept = model[pair[0]] + model[pair[1]]
    similarities = model.similar_by_vector(generated_concept)
    print(f"Most similar tokens to '{pair[0]}' + '{pair[1]}' (for '{concept}')")
    df = pd.DataFrame(similarities, columns=['WORD', 'SCORE'])
    display(df)

Most similar tokens to 'hate' + 'marriage' (for 'love')


,WORD,SCORE
0,hate,0.879536
1,appeal,0.845385
2,Armagh.,0.827039
3,explication,0.822837
4,Bunny.,0.817900
5,dicto,0.816709
6,"palaces,",0.814199
7,denominations,0.812224
8,waters.,0.807174
9,fop's,0.806112


Not bad! Our target concept isn't the most similar word for either of these examples, but it's in the top 10.

Most famously, word embeddings enable quasi-logical reasoning. Though, as we mentioned earlier, relationships between antonyms and synoyms do not necessarily map to a vector space, certain analogies do – at least under the right circumstances, and with particular training data. The logic here is that we identify a relationship between two words and we subtract one of those words' vectors from the other. To that new vector we add in a vector for a target word, which forms the analogy. Querying for the word closest to this modified vector should produce a similar relation between the result and the target word as that between the original pair.

Here, we ask: "strong is to stronger what clear is to X?"

In [ ]:
analogies = model.most_similar(positive=['stronger', 'clear'], negative=['strong'])
display(pd.DataFrame(analogies, columns=['WORD', 'SCORE']))
print("Ideal target: 'clearer'")

And here, we ask: "Paris is to France what Berlin is to X"?

In [ ]:
analogies = model.most_similar(positive=['france', 'berlin'], negative=['paris'])
display(pd.DataFrame(analogies, columns=['WORD', 'SCORE']))
print("Ideal target: 'Germany'")

Both of the above produce compelling results, though your mileage may vary. Consider the following: "arm is to hand what leg is to X?"

In [ ]:
analogies = model.most_similar(positive=['hand', 'leg'], negative=['arm'])
display(pd.DataFrame(analogies, columns=['WORD', 'SCORE']))
print("Ideal target: 'foot'")

Importantly, these results are always going to be specific to the data on which a model was trained. Claims made on the basis of word embeddings that aspire to general linguistic truths would be treading on shaky ground here.

Document similarity
------------------------

While the above word relationships are relatively abstract (and any such findings therefrom should be couched accordingly), we can ground them with a concrete task. In this final section, we use GloVe embeddings to encode our corpus documents. This involves associating a word vector for each token in an obituary. Of course, GloVe has not been trained on the obituaries, so there may be important differences in token behavior between that model and the corpus; but we can assume that the general nature of GloVe will give us a decent sense of the overall feature space of the corpus. The result will be an enriched representation of each document, the nuances of which may better help us identify things like similarities between obituaries in our corpus.

The other consideration for using GloVe with our specific corpus concerns the out-of-vocabulary words we've already discussed. Before we can encode our documents, we need to filter out tokens for which GloVe has no representation. We can do so by referencing the `in_glove` set we produced above.

```{margin} What this loop does
For every obituary:

1. Create a new list to hold the tokens we want to keep
2. Go through each of the tokens and check whether they are in GloVe
3. Append in-vocabulary tokens to the new list
4. Once we've checked all tokens, append them to a new corpus list
```

In [33]:
pruned = []
for doc in corpus:
    keep = []
    for token in doc:
        if token in in_glove:
            keep.append(token)
    pruned.append(keep)

### Encoding

Time to encode. This is an easy operation. All we need to do is run the list of document's tokens directly into the model object and `gensim` will encode each accordingly. The result will be an `(n, 200)` array, where `n` is the number of tokens we passed to the model; each one will have 200 dimensions.

But if we kept this array as is, we'd run into trouble. Matrix operations often require identically shaped representations, so documents with different lengths would be incomparable. To get around this, we take the mean of all the vectors in a document. The result is a 200-dimension vector that stands as a general representation of a document.

In [34]:
doc_embeddings = [np.mean(model[doc], axis=0) for doc in pruned]
doc_embeddings = np.array(doc_embeddings)

Let's quickly check our work.

In [ ]:
print(
    f"Shape of an encoded document: {model[pruned[0]].shape}",
    f"\nShape of an encoded document after taking its mean embedding: {doc_embeddings[0].shape}"
)

From here, we can treat these embeddings almost as if they represented words. Let's plot our obituaries accordingly. Take a look around at this and see what you can find. As a starting point, you might focus on that cluster of nodes right in the middle of the graph, toward the top. All the obituaries there are for sports players – they're even broken out by sport (baseball players are on the right).

In [ ]:
vis_data = prepare_vis_data(doc_embeddings, manifest['NAME'])

alt.Chart(vis_data).mark_circle(size=30).encode(
    x='X',
    y='Y',
    tooltip='LABEL'
).properties(
    height=650,
    width=650
).interactive()

### Clustering

The document embeddings seem to be partitioned into different clusters. We'll end by using a hierarchical clusterer to see if we can further specifiy these clusters. This involves loading the `AgglomerativeClustering` object from `scikit-learn` and fitting it to our document embeddings. Hierarchical clustering requires us to predefine the number of clusters we'd like to generate. In this case, we'll go with 18.

```{margin} Why this number of clusters?
We grid searched different numbers and measured the results with a [silhouette coefficient].

[silhouette coefficient]: https://towardsdatascience.com/silhouette-coefficient-validating-clustering-techniques-e976bb81d10c
```

In [37]:
from sklearn.cluster import AgglomerativeClustering

n_clusters = 18
agg = AgglomerativeClustering(n_clusters=n_clusters).fit(doc_embeddings)

Now we can assign the clusterer's predicted labels to the dataframe that contains our visualization data and re-plot the results.

In [ ]:
vis_data['CLUSTER'] = agg.labels_ + 1

alt.Chart(vis_data).mark_circle(size=30).encode(
    x='X',
    y='Y',
    tooltip=['LABEL', 'CLUSTER'],
    color='CLUSTER:N'
).properties(
    height=650,
    width=650
).interactive()

Once again, take a look around and see what you can find. These clusters seem to be both detailed and nicely partitioned, bracketing off, for example, classical musicians and composers (cluster 7) from jazz and popular musicians (cluster 11).

```{margin} What this loop does
For each cluster:

1. Subset the visualization data with that cluster and get all the people
2. Convert the column into string and then split that string into chunks with `textwrap`
3. Print to screen
```

In [ ]:
import textwrap

for k in [7, 11]:
    people = vis_data[vis_data['CLUSTER']==k]['LABEL']
    people = ', '.join(person for person in people)
    people = textwrap.wrap(people, 80)
    print(f"Cluster: {k:>2}\n-----------")
    for entry in people:
        print(entry)
    print("\n")

Consider further cluster 6, which seems to be about famous scientists.

```{margin} What's going on with "Martian Theory"?
It appears this is actually Percival Lowell, an astronomer (he did, however, advance the idea that Mars is inhabited). Apparently our metadata is a little messy!
```

In [ ]:
for person in vis_data[vis_data['CLUSTER']==6]['LABEL']:
    print(person)

There are, however, some interestingly noisy clusters, like cluster 13. With people like Queen Victoria and William McKinley in this cluster, it at first appears to be about national leaders of various sorts, but the inclusion of others like Al Capone (the ganster) and Ernie Pyle (a journalist) complicate this. If you take a closer look, what really seems to be tying these obituaries together is war. Nearly everyone here was involved in war in some fashion or another – save for Capone, whose inclusion makes for strange bedfellows.

In [ ]:
for person in vis_data[vis_data['CLUSTER']==13]['LABEL']:
    print(person)

Depending on your task, these detailed distinctions may not be so desirable. But for us, the document embeddings provide a wonderfully nuanced view of the kinds of people in the obituaries. From here, further exploration might involve focusing on misfits and outliers. Why, for example, is Capone in cluster 13? Or why is Lou Gehrig all by himself in his own cluster? Of course, we could always recluster this data, which would redraw such groupings, but perhaps there is something indeed significant about the way things are divided up as they stand. Word embeddings help bring us to a point where we can begin to undertake such investigations – what comes next depends on which questions we want to ask.